#유사도기반 상위 N개 아이템을 반환하는 메소드 작성

##태그기반 유사도 활용

In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
dir_path = "/content/drive/MyDrive/yeoreodigm/data_files"
attraction = pd.read_csv(dir_path + "/attraction_db_0716.csv")
course = pd.read_csv(dir_path + '/07.18_course_db_with_cluster.csv',index_col=0)

In [21]:
attraction.head(3)

,Unnamed: 0,id,title,address,introduction,latitude,longitude,dial_num,type,region1,...,image_url,score,nature,outdoor,fatigue,sea,walking,exciting,day,culture
0,0,CONT_000000000500281,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.408450,126.639760,064-900-8800,관광지,제주시,...,https://api.cdn.visitjeju.net/photomng/imgpath...,1276524.0,2,2,1,1,3,1,0,0
1,1,CONT_000000000500477,우도,제주특별자치도 제주시 우도면 삼양고수물길 1,소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬,33.519490,126.951090,064-728-1527,관광지,섬 속의 섬,...,https://api.cdn.visitjeju.net/photomng/imgpath...,1188600.0,3,3,3,1,3,0,3,0
2,2,CONT_000000000500349,성산일출봉,제주특별자치도 서귀포시 성산읍 일출로 284-12,"바다위에 우뚝 솟아난 수성화산·유네스코 세계자연유산, 천연기념물 제420호, 올레1코스",33.462147,126.936424,064-783-0959,관광지,서귀포시,...,https://api.cdn.visitjeju.net/photomng/imgpath...,1027366.0,3,3,3,0,3,0,3,1


In [22]:
attraction.drop(columns=['Unnamed: 0'],inplace = True)

In [25]:
course.head(5)

,Package_Id,Package Title,serial path(no airport),path_only_attraction,len,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster
0,DCKC2TKO4,[청주출발]연합패키지 제주를 반하다 3일_관광호텔 화순곶자왈+서커스+석예원한방족욕+...,"수목원테마파크,카멜리아힐,성읍민속마을,아쿠아플라넷","수목원테마공원,카멜리아힐,성읍민속마을,아쿠아플라넷제주",4,1.250000,2.000000,1.250000,1.250000,2.750000,1.000000,1.750000,0.250000,3
1,DCKC2TKO3,[청주출발]연합패키지 노팁/노옵션 제주 3일_관광호텔 산방산유람선+ 화순곶자왈+서커...,"애월한담해안도로,용두암,카멜리아힐,수목원테마파크,산방산유람선,성읍민속마을,아쿠아플라넷","애월한담해안산책로,용두암,카멜리아힐,수목원테마공원,산방산,성읍민속마을,아쿠아플라넷제주",7,1.714286,2.428571,1.285714,1.428571,2.571429,0.571429,2.285714,0.428571,3
2,DCKP2TKO4,[부산출발_※3일차 석식 전복죽포함※]연합패키지 제주를 반하다 3일_관광호텔 화순곶...,"애월한담해안도로,수목원테마파크,카멜리아힐,성읍민속마을,아쿠아플라넷","애월한담해안산책로,수목원테마공원,카멜리아힐,성읍민속마을,아쿠아플라넷제주",5,1.400000,2.200000,1.200000,1.400000,2.800000,0.800000,2.000000,0.200000,3
3,DCPC2TKO5,"[청주출발]""모두 알뜰 패키지"" 제주 관광호텔 2박3일 비밀의 그림자+ 웰빙 족욕","용두암,제주레저승마,성읍민속마을","용두암,제주승마공원,성읍민속마을",3,2.000000,3.000000,1.666667,1.666667,1.666667,1.000000,3.000000,0.666667,6
4,DCPC2TKO09,[청주출발]◈감귤타르트증정/코시롱한나들이◈아쿠아플라넷+유람선-제주 로베로관광호텔 2...,"용두암,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷","용두암,산방산,제주승마공원,성읍민속마을,아쿠아플라넷제주",5,1.600000,2.400000,1.600000,1.400000,2.000000,0.800000,2.800000,0.600000,3


In [12]:
#attraction.drop(columns=['Unnamed: 0'],inplace = True)
course.drop(columns=['Unnamed: 0','index','Duration','attraction list'],inplace=True)

In [26]:
course.iloc[:2,5:]

,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster
0,1.250000,2.000000,1.250000,1.250000,2.750000,1.000000,1.750000,0.250000,3
1,1.714286,2.428571,1.285714,1.428571,2.571429,0.571429,2.285714,0.428571,3


In [16]:
sample_data = course.sample().iloc[0,5:]
sample_data

nature      1.714286
outdoor     2.142857
fatigue     1.428571
sea         0.571429
walking     1.571429
exciting    0.571429
day         2.428571
culture          1.0
Name: 3267, dtype: object

In [41]:
course.index[course['Package_Id']=="DCKC2TKO4"]

Int64Index([0], dtype='int64')

In [50]:
test_similarity = cosine_similarity(course.iloc[:100,5:])
first_sim = list(test_similarity[0])
test_similarity

array([[1.        , 0.98721902, 0.99754333, ..., 0.9752201 , 0.92688742,
        0.92240536],
       [0.98721902, 1.        , 0.9944776 , ..., 0.99329093, 0.92479811,
        0.92012198],
       [0.99754333, 0.9944776 , 1.        , ..., 0.98318864, 0.92348384,
        0.91894902],
       ...,
       [0.9752201 , 0.99329093, 0.98318864, ..., 1.        , 0.92666059,
        0.92110233],
       [0.92688742, 0.92479811, 0.92348384, ..., 0.92666059, 1.        ,
        0.99937911],
       [0.92240536, 0.92012198, 0.91894902, ..., 0.92110233, 0.99937911,
        1.        ]])

In [61]:
similarity = cosine_similarity(course.iloc[:100,5:])

In [62]:
similarity.shape

(100, 100)

In [51]:
max_val = max(first_sim[1:])
min_val = min(first_sim)
max_idx = first_sim.index(max_val)
min_idx = first_sim.index(min_val)
print(max_val,min_val)
print(max_idx,min_idx)

0.9975433321003042 0.7780317569693779
2 20


###코스간 유사도

In [148]:
def recommend_course_list(df,course_id,top):
  target_course_index = df[df['Package_Id']==course_id].index.values
  print(target_course_index)
  total_similarity = cosine_similarity(df.iloc[:,5:])
  print(total_similarity.shape)
  similarity = total_similarity[target_course_index]
  similarity = list(similarity[0])
  
  
  print('similarity : ',similarity)
  
  similarity_with_index = [(val,idx) for idx,val in enumerate(similarity)]
  similarity_with_index.sort(reverse = True) 
  print("similarity_with_index : ",similarity_with_index)


  result_index = [i[1] for i in similarity_with_index]
  print("result_index : ",result_index)

  #자기 자신은 제외 (1일것임)
  me=0
  top_N_result = result_index[me:top+1] #자기 자신은 제외하려면 me -> me+1로 해주면 됨
  print(top_N_result)
  

  result = df.iloc[top_N_result]
  return result

In [149]:
recommend_course_list(course,'DCPC2TKO09',10)

[4]
(3811, 3811)
similarity :  [0.9681217950530232, 0.9886011157359574, 0.9756687144069505, 0.95567524893373, 1.0, 0.9061239215285767, 0.8314898724158113, 0.8330031198947753, 0.9362649022823527, 0.8051806099703039, 0.8206778075714984, 0.8381462692870996, 0.8331332310360176, 0.8259741463013914, 0.991312724046893, 0.8383453131674499, 0.8104011372110024, 0.9841319087139556, 0.9154020438353483, 0.8984086018676695, 0.8524345742680899, 0.8561034025537899, 0.9870747120447587, 0.9903347029211738, 0.8089046804700532, 0.941179645427948, 0.9756687144069505, 0.982452189246476, 0.9759484086447008, 0.9148195416803808, 0.9914069378161317, 0.9808454044004119, 0.9889044518194665, 0.9919593747195401, 0.9927069698695993, 0.9888233544458676, 0.9846137264351688, 0.9846137264351688, 0.990813592150974, 0.8145255297582518, 0.988688255220595, 0.9770532597937939, 0.967628922926592, 0.9681312764486085, 0.8019084092049681, 0.9071989547099565, 0.8478109344846423, 0.9148200207809055, 0.9324310187326209, 0.979743801

,Package_Id,Package Title,serial path(no airport),path_only_attraction,len,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster
4,DCPC2TKO09,[청주출발]◈감귤타르트증정/코시롱한나들이◈아쿠아플라넷+유람선-제주 로베로관광호텔 2...,"용두암,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷","용두암,산방산,제주승마공원,성읍민속마을,아쿠아플라넷제주",5,1.600000,2.400000,1.600000,1.400000,2.000000,0.800000,2.800000,0.600000,3
810,50d34502d2fcba7a,5월 황금연휴! 아기와 함께 떠나는 제주도 여행,"우리집,만복이네김밥집,베이비엔카 서부지점,이호테우목마등대,애월 해안도로,탐라대홍가시...","이호테우말등대,애월해안도로,한라수목원,본태박물관,제주항공우주박물관,점보빌리지,노리매...",13,1.615385,2.230769,1.384615,1.461538,1.923077,0.846154,2.846154,0.615385,3
1323,91604c750e0acba6,제주도 반값여행,"이호테우목마등대,몽상드애월,춘심이네 본점,데이즈호텔 제주서귀포 오션,데이즈호텔 제주...","이호테우말등대,이중섭거리,쇠소깍,우도,성산일출봉,아쿠아플라넷제주,월정리해변",7,1.714286,2.428571,1.571429,1.428571,2.142857,1.000000,2.857143,0.428571,3
3430,3c7eddfcdaf0a823,2018제주,"마라도 가는 여객선,마라도,마라도 가는 여객선,하얏트 리젠시 제주,쌍둥이 횟집,이중...","마라도,이중섭거리,새연교,넥슨컴퓨터박물관,애월한담해안산책로,신창풍차해안도로,성산일출...",10,1.700000,2.600000,1.400000,1.400000,2.000000,0.900000,2.900000,0.600000,3
3186,0ae687efaede18b6,제주도3박4일 홀로 힐링기,"함덕 서우봉 해변,책닦는 남자,비자림,김녕성세기해변,너는파라다이스길리,이찌마루,김영...","비자림,김녕성세기해변,김영갑갤러리두모악,쇠소깍,정방폭포,이중섭거리,용두암,에코랜드테...",9,1.777778,2.444444,1.555556,1.444444,2.000000,0.888889,2.555556,0.666667,3
2160,c8560a61d2acf857,힐링 제주,"애월 해안도로,리치망고 애월본점,곽지해수욕장,금능 해수욕장야영장,토리의꿈게스트하우스...","애월해안도로,곽지해수욕장,금능해수욕장,성이시돌목장,방주교회,본태박물관,카멜리아힐,물...",16,1.750000,2.375000,1.687500,1.562500,2.125000,0.812500,2.562500,0.500000,3
360,d7f11311b9d7b9a7,♥ 4박5일 희수 ? 정인 제주도 여행 ♥,"애월초등학교 더럭분교장,이호테우해변,리치망고 애월본점,아라파파,성이시돌목장,거멍국수...","이호테우해수욕장,성이시돌목장,카멜리아힐,방주교회,유민미술관,유민미술관,한라승마장,마...",13,1.769231,2.230769,1.615385,1.538462,1.923077,1.076923,2.692308,0.615385,3
670,efcc8461c7f99aef,3박4일,"카멜리아 힐,다빈치 뮤지엄,산방산 탄산온천,청수곶자왈,일레인호텔,일레인호텔,천지연폭...","카멜리아힐,산방산탄산온천,청수곶자왈,천지연폭포,이중섭거리,본태박물관,쇠소깍,아쿠아플...",11,1.636364,2.090909,1.454545,1.272727,2.000000,0.636364,2.454545,0.454545,3
2281,a1773899828708d3,제주여행,"로베로 호텔,동진식당,제주바다,애월 해안도로,하와유제주,고내포구,애월초등학교 더럭분...","제주다,애월해안도로,고내리포구,협재해수욕장,협재해수욕장,월정리해변,성산일출봉,혼인지...",11,1.909091,2.636364,1.545455,1.727273,2.090909,0.909091,2.909091,0.454545,3
540,9f79ed909e7fd80d,남팅구랑,"동문시장,협재해수욕장,애월초등학교 더럭분교장,요리하는 목수,제주 러브랜드,라움호텔,...","협재해수욕장,제주러브랜드[휴업중],제주조천스위스마을,노루생태관찰원,헬로키티아일랜드,...",10,1.600000,2.300000,1.600000,1.600000,2.000000,1.100000,2.700000,0.400000,3


In [120]:
test_list = [5.5,4.4,3.3,2.2,1.1]
temp = [(val,idx) for idx,val in enumerate(test_list)]
temp.sort()
print(temp)


[(1.1, 4), (2.2, 3), (3.3, 2), (4.4, 1), (5.5, 0)]


###여행지 유사도

In [137]:
attraction.head(10)

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,image_url,score,nature,outdoor,fatigue,sea,walking,exciting,day,culture
0,CONT_000000000500281,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.408450,126.639760,064-900-8800,관광지,제주시,조천,https://api.cdn.visitjeju.net/photomng/imgpath...,1276524.0,2,2,1,1,3,1,0,0
1,CONT_000000000500477,우도,제주특별자치도 제주시 우도면 삼양고수물길 1,소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬,33.519490,126.951090,064-728-1527,관광지,섬 속의 섬,우도,https://api.cdn.visitjeju.net/photomng/imgpath...,1188600.0,3,3,3,1,3,0,3,0
2,CONT_000000000500349,성산일출봉,제주특별자치도 서귀포시 성산읍 일출로 284-12,"바다위에 우뚝 솟아난 수성화산·유네스코 세계자연유산, 천연기념물 제420호, 올레1코스",33.462147,126.936424,064-783-0959,관광지,서귀포시,성산,https://api.cdn.visitjeju.net/photomng/imgpath...,1027366.0,3,3,3,0,3,0,3,1
3,CONT_000000000500270,비자림,제주특별자치도 제주시 구좌읍 비자숲길 55,천년의 세월이 녹아든 신비로움 가득한 비자림은 500~800년생 비자나무들이 자생하...,33.491318,126.811386,064-710-7912,관광지,제주시,구좌,https://api.cdn.visitjeju.net/photomng/imgpath...,916549.0,2,3,3,1,3,0,3,0
4,CONT_000000000500618,천지연폭포,제주특별자치도 서귀포시 남성중로 2-15,"하늘과 땅이 만나서 이룬 연못, \r\n· 천연기념물 제163호\r\n· 천연기념물...",33.244717,126.559820,064-733-1528,관광지,서귀포시,서귀포시내,https://api.cdn.visitjeju.net/photomng/imgpath...,645261.0,3,3,1,1,1,0,0,0
5,CONT_000000000500182,만장굴,제주특별자치도 제주시 구좌읍 만장굴길 182,"세계적인 규모의 용암동굴, 유네스코 지정 세계자연유산",33.554819,126.796902,064-710-7903,관광지,제주시,구좌,https://api.cdn.visitjeju.net/photomng/imgpath...,612892.0,3,2,1,1,2,1,3,1
6,CONT_000000000500309,새별오름,제주특별자치도 제주시 애월읍 봉성리 산 59-8,제주 서부 애월에 위치해 있으며 억새가 아름다운 오름,33.367230,126.357110,064-728-2752,관광지,제주시,애월,https://api.cdn.visitjeju.net/photomng/imgpath...,462211.0,3,2,0,1,0,0,2,1
7,CONT_000000000500697,협재해수욕장,제주특별자치도 제주시 한림읍 한림로 329-10,"비양도, 은모래, 바다가 그려낸 그림같이 아름다운 해변",33.393845,126.239716,064-728-3981,관광지,제주시,한림,https://api.cdn.visitjeju.net/photomng/imgpath...,444882.0,2,3,2,3,2,2,3,0
8,CONT_000000000500343,섭지코지,제주특별자치도 서귀포시 성산읍 섭지코지로 107,아름다운 해안이 일품인 섭지,33.430578,126.927769,064-740-6000,관광지,서귀포시,성산,https://api.cdn.visitjeju.net/photomng/imgpath...,403859.0,3,3,3,1,3,0,3,0
9,CNTS_000000000001195,카멜리아힐,제주 서귀포시 안덕면 병악로 166,"카멜리아힐은 동양에서 가장 큰 동백 수목원으로 토종 동백부터 아기 동백, 유럽 동백...",33.289135,126.370110,064-792-0088,관광지,서귀포시,안덕,https://api.cdn.visitjeju.net/photomng/imgpath...,387309.0,2,2,1,1,3,1,2,0


In [134]:
attraction.drop(columns=['Unnamed: 0'],inplace=True)

In [135]:
attraction.columns

Index(['id', 'title', 'address', 'introduction', 'latitude', 'longitude',
       'dial_num', 'type', 'region1', 'region2', 'image_url', 'score',
       'nature', 'outdoor', 'fatigue', 'sea', 'walking', 'exciting', 'day',
       'culture'],
      dtype='object')

In [145]:
attraction.iloc[:,12:].sample()

,nature,outdoor,fatigue,sea,walking,exciting,day,culture
106,3,3,2,1,1,2,3,0


In [146]:
def recommend_attraction_list(df,attraction_id,top):
  target_attraction_index = df[df['id']==attraction_id].index.values
  print(target_attraction_index)
  total_similarity = cosine_similarity(df.iloc[:,12:])
  print(total_similarity.shape)
  similarity = total_similarity[target_attraction_index]
  similarity = list(similarity[0])
  
  
  print('similarity : ',similarity)
  
  similarity_with_index = [(val,idx) for idx,val in enumerate(similarity)]
  similarity_with_index.sort(reverse = True) 
  print("similarity_with_index : ",similarity_with_index)


  result_index = [i[1] for i in similarity_with_index]
  print("result_index : ",result_index)

  #자기 자신은 제외 (1일것임)
  top_N_result = result_index[:top+1]
  print(top_N_result)
  

  result = df.iloc[top_N_result].sort_values(by='score',ascending=False)
  return result

In [147]:
test_attraction_id = "CONT_000000000500697"
recommend_attraction_list(attraction,test_attraction_id,20)

[7]
(1089, 1089)
similarity :  [0.7842934903910445, 0.8769028133059114, 0.8094487507439182, 0.8812021902246507, 0.732114158203694, 0.8909536742235271, 0.7346969999683854, 1.0, 0.8769028133059114, 0.902730569228801, 0.7879092800517907, 0.990507269616327, 0.964772650936774, 0.8094487507439182, 0.8248628195623473, 0.8892118276421004, 0.9279339519604581, 0.9345030284511884, 0.914991421995628, 0.732114158203694, 0.8183932073645682, 0.8641585652180931, 0.9767441860465118, 0.9050184656361278, 0.990507269616327, 0.7879092800517907, 0.8094487507439182, 0.7990858564510239, 0.7696825713954514, 0.9061847548617584, 0.990507269616327, 0.9153651166903974, 0.669374346314434, 0.990507269616327, 0.79570065658118, 0.8198127976897006, 0.9177105704032729, 0.8198127976897006, 0.929130778103323, 0.9345030284511884, 0.9345030284511884, 0.9153651166903974, 0.9345030284511884, 0.8094487507439182, 0.9565007145952775, 0.9153651166903974, 0.9056241156018814, 0.8097533639902196, 0.9025419790150204, 0.82175417923724

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,image_url,score,nature,outdoor,fatigue,sea,walking,exciting,day,culture
7,CONT_000000000500697,협재해수욕장,제주특별자치도 제주시 한림읍 한림로 329-10,"비양도, 은모래, 바다가 그려낸 그림같이 아름다운 해변",33.393845,126.239716,064-728-3981,관광지,제주시,한림,https://api.cdn.visitjeju.net/photomng/imgpath...,444882.0,2,3,2,3,2,2,3,0
11,CONT_000000000500693,함덕해수욕장,제주특별자치도 제주시 조천읍 조함해안로 519-10,에매랄드처럼 빛나는 '한국의 몰디브',33.543790,126.668846,064-728-3989,관광지,제주시,조천,https://api.cdn.visitjeju.net/photomng/imgpath...,355426.0,2,3,2,3,3,2,3,0
24,CONT_000000000500083,김녕성세기해변,제주특별자치도 제주시 구좌읍 구좌해안로 237,너럭바위 위에 생긴 김녕해수욕장,33.557926,126.759980,064-728-3987,관광지,제주시,구좌,https://api.cdn.visitjeju.net/photomng/imgpath...,185541.0,2,3,2,3,3,2,3,0
30,CONT_000000000500056,곽지해수욕장,제주특별자치도 제주시 애월읍 애월원당길 (곽지리),시원한 용천수를 뿜어내는 노천탕이 있는 곳,33.451220,126.305800,064-728-3985,관광지,제주시,애월,https://api.cdn.visitjeju.net/photomng/imgpath...,160888.0,2,3,2,3,3,2,3,0
33,CONT_000000000500079,금능해수욕장,제주특별자치도 제주시 한림읍 금능길 119-10,제주시 한림읍에 위치한 금능해수욕장은 서쪽의 인기 명소인 협재해수욕장과 바로 이어져...,33.390003,126.235910,064-728-3983,관광지,제주시,한림,https://api.cdn.visitjeju.net/photomng/imgpath...,147218.0,2,3,2,3,3,2,3,0
55,CONT_000000000500604,중문색달해수욕장,제주특별자치도 서귀포시 중문관광로72번길 29-51,"제주올레 8코스, 여유롭고 이국적인 해수욕장",33.244990,126.411590,064-760-4993,관광지,서귀포시,중문,https://api.cdn.visitjeju.net/photomng/imgpath...,115910.0,2,3,2,3,3,2,3,0
311,CNTS_000000000022542,판포포구,제주특별자치도 제주시 한경면 판포리 2877-3,서귀포 끝자락에 있는 깨끗한 물색을 자랑하는 작은 포구,33.365370,126.199650,--,관광지,제주시,한경,https://api.cdn.visitjeju.net/photomng/imgpath...,20798.0,3,3,2,3,2,2,3,0
406,CNTS_000000000019892,스쿠버스쿨,제주특별자치도 제주시 서해안로 301,"탐험, 도전! 도시를 떠나 자신을 찾아 도전하라",33.508968,126.477676,064-713-2711,관광지,제주시,제주시내,https://api.cdn.visitjeju.net/photomng/imgpath...,14539.0,3,3,2,3,2,2,3,0
887,CNTS_000000000020935,동일리포구,제주특별자치도 서귀포시 대정읍 동일리,정겨움 가득한 마을과 동일리포구,33.225437,126.241330,--,관광지,서귀포시,대정,https://api.cdn.visitjeju.net/photomng/imgpath...,3991.0,1,2,1,2,1,1,2,0
896,CNTS_000000000021522,공천포구,제주특별자치도 서귀포시 남원읍 신례리,검은 돌과 검은 모래가 가득한 바다,33.263120,126.641650,--,관광지,서귀포시,남원,https://api.cdn.visitjeju.net/photomng/imgpath...,3901.0,1,2,1,2,1,1,2,0


In [130]:
import numpy as np
doc1 = np.array([[0,1,2,3]])
doc1.shape
print(doc1)

[[0 1 2 3]]
